# Tomato Ripeness Detection
Using transfer learning, creating a model that detects and classifies if a tomato is ripe

In [59]:
import tensorflow as tf
import numpy as np
import pandas as pd
import kaggle
from pylabel import importer

#### Loading Data

In [63]:
#Specify path to the coco.json file
path_to_annotations = "/content/tomatod/annotations/tomatOD_train.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = "/content/tomatod/images/train"

#Import the dataset into the pylable schema
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
type(dataset.df)

pandas.core.frame.DataFrame